In [ ]:
#code for Mage

import io
import requests
import docx
def clean_line(line):
    line = line.strip()
    line = line.strip('\uFEFF')
    return line

def read_faq(file_id):
    url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    
    response = requests.get(url)
    response.raise_for_status()
    
    with io.BytesIO(response.content) as f_in:
        doc = docx.Document(f_in)

    questions = []

    question_heading_style = 'heading 2'
    section_heading_style = 'heading 1'
    
    heading_id = ''
    section_title = ''
    question_title = ''
    answer_text_so_far = ''
     
    for p in doc.paragraphs:
        style = p.style.name.lower()
        p_text = clean_line(p.text)
    
        if len(p_text) == 0:
            continue
    
        if style == section_heading_style:
            section_title = p_text
            continue
    
        if style == question_heading_style:
            answer_text_so_far = answer_text_so_far.strip()
            if answer_text_so_far != '' and section_title != '' and question_title != '':
                questions.append({
                    'text': answer_text_so_far,
                    'section': section_title,
                    'question': question_title,
                })
                answer_text_so_far = ''
    
            question_title = p_text
            continue
        
        answer_text_so_far += '\n' + p_text
    
    answer_text_so_far = answer_text_so_far.strip()
    if answer_text_so_far != '' and section_title != '' and question_title != '':
        questions.append({
            'text': answer_text_so_far,
            'section': section_title,
            'question': question_title,
        })

    return questions

@data_loader
def load_data(*args, **kwargs):
    faq_documents = {
    'llm-zoomcamp': '1T3MdwUvqCL3jrh3d3VCXQ8xE0UqRzI3bfgpfBq3ZWG0',
    }
    documents = []

    for course, file_id in faq_documents.items():
        print(course)
        course_documents = read_faq(file_id)
        documents.append({'course': course, 'documents': course_documents})
    print(len(documents))
    return documents




In [ ]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

@transformer
def transform(data, *args, **kwargs):
    print(type(data), len(data))
    documents = []
    if type(data) == list and len(data) == 1: data = data[0]
    
    

    for doc in data['documents']:
        doc['course'] = data['course']
        # previously we used just "id" for document ID
        doc['document_id'] = generate_document_id(doc)
        documents.append(doc)

    print(len(documents))

    return documents
    




In [ ]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

@transformer
def transform(data, *args, **kwargs):
    print(type(data), len(data))
    documents = []
    if type(data) == list and len(data) == 1: data = data[0]
    
    

    for doc in data['documents']:
        doc['course'] = data['course']
        # previously we used just "id" for document ID
        doc['document_id'] = generate_document_id(doc)
        documents.append(doc)

    print(len(documents))

    return documents
    




In [ ]:
from typing import Dict, List, Union

import numpy as np
from elasticsearch import Elasticsearch



@data_loader


def search(*args, **kwargs):
    connection_string = kwargs.get('connection_string', 'http://Elasticsearch:9200')
    es_client = Elasticsearch(connection_string)
    indices = es_client.indices.get_alias(index="*")
    print(indices)
    #index_name = kwargs.get('index_name', 'documents_20240819_5838')
    index_name = 'documents_20240819_3230'
    query_text = "When is the next cohort?"
    top_k = kwargs.get('top_k', 5)
    response = es_client.search(
            index=index_name,
            body={
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["question", "text"],
                            "type": "best_fields"
                        }
                    }
                }
            }
        }
    )    
            
        
    
    
    

    
    

    print(response)
    print([hit['_source']['document_id'] for hit in response['hits']['hits']]
)
    return [hit['_source']['document_id'] for hit in response['hits']['hits']]


